# Example Usage of EQTransformer from seisbench
This notebook is mostly an uncommented version of the “03a_training_phasenet” notebook from the seisbench repository.

In [1]:
import seisbench.generate as sbg
import seisbench.models as sbm

from seisbench.data import WaveformDataset
from seisbench.models import EQTransformer
from seisbench.util import worker_seeding

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader


Load the 100Samples dataset from the EQTransformer repository but transformed into seisbench-compatible format.

In [2]:
example_data = WaveformDataset("data/STEAD/example/seisbench")


Instantiate a version of seisbench’s EQTransformer

In [3]:
eqt = EQTransformer()
eqt = sbm.EQTransformer().from_pretrained("original")


This part is slightly weird, but the *data* contains information about what is training, validation, and test, already.
In this case, I just randomly designated 81 samples as training data, 9 as dev/validation data, and the remaining 10 as test data; see also the “Convert Example Data to Seisbench Format” notebook.

In [4]:
train, dev, test = example_data.train_dev_test()


Rename the label columns.

In [5]:
phase_dict = {
    "trace_p_arrival_sample": "P",
    "trace_s_arrival_sample": "S"
}


Perform (minimal) data augmentation.  I assume (but did not test) that `ChangeDtype` is necessary for the code to run; `ProbabilisticLabeller` provides the probability curves below, but I do not know how they are calculated.

In [6]:
train_generator = sbg.GenericGenerator(train)
dev_generator = sbg.GenericGenerator(train)

augmentations = [
    sbg.ChangeDtype(np.float32),
    sbg.ProbabilisticLabeller(label_columns=phase_dict, sigma=30, dim=0)
]

train_generator.add_augmentations(augmentations)
dev_generator.add_augmentations(augmentations)


In [7]:
batch_size = 100 #
num_workers = 1  # The number of threads used for loading data

train_loader = DataLoader(train_generator, batch_size=batch_size, shuffle=True, num_workers=num_workers, worker_init_fn=worker_seeding)
dev_loader = DataLoader(dev_generator, batch_size=batch_size, shuffle=False, num_workers=num_workers, worker_init_fn=worker_seeding)


### Laplace

In [8]:
from laplace import Laplace


In [9]:
la = Laplace(eqt, 'regression', subset_of_weights='all', hessian_structure='diag')


In [10]:
la.fit(train_loader)


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


/home/robot/anaconda3/envs/dslab/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([81, 3, 6000])) that is different to the input size (torch.Size([3, 6000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


AssertionError: BackPACK extension expects a backpropagation quantity but it is None. Module: Component order:	ZNE
SeisBench model		EQTransformer

EQTransformer(
  (encoder): Encoder(
    (convs): ModuleList(
      (0): Conv1d(3, 8, kernel_size=(11,), stride=(1,), padding=(5,))
      (1): Conv1d(8, 16, kernel_size=(9,), stride=(1,), padding=(4,))
      (2): Conv1d(16, 16, kernel_size=(7,), stride=(1,), padding=(3,))
      (3): Conv1d(16, 32, kernel_size=(7,), stride=(1,), padding=(3,))
      (4): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (5): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (pools): ModuleList(
      (0-6): 7 x MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (res_cnn_stack): ResCNNStack(
    (members): ModuleList(
      (0-3): 4 x ResCNNBlock(
        (dropout): SpatialDropout1d(
          (dropout): Dropout2d(p=0.1,